<a href="https://colab.research.google.com/github/vistec-AI/colab/blob/master/OSCAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

เขาเลือกจาก common crawl มาอีกที [OSCAR](https://traces1.inria.fr/oscar/)

## Import

In [0]:
import re
import pandas as pd
import plotly.express as px
from collections import Counter

In [0]:
!pip install pythainlp --pre

     |████████████████████████████████| 11.1MB 3.1MB/s 
     |████████████████████████████████| 1.4MB 44.2MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434677 sha256=4962865dffa46f14843f217a8b229989219a626484181f8ee71f7181c1eaff25
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
  Found existing installation: requests 2.21.0
    Uninstalling requests-2.21.0:
      Successfully uninstalled requests-2.21.0


In [0]:
#@title icu
!pip install pyicu
import icu

def icu_gen_words(text):
  it = icu.BreakIterator.createWordInstance(icu.Locale("th"))
  it.setText(text)
  start = it.first()
  for end in it:
    yield text[start:end]
    start = end

def icu_segment(text):
  return list(icu_gen_words(text))

## Download

In [0]:
# เขาแนะนำตัวนี้ น่าจะเร็วกว่า gzip ปกติ เพราะ parallel
!apt install pigz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  pigz
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 57.4 kB of archives.
After this operation, 259 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 pigz amd64 2.4-1 [57.4 kB]
Fetched 57.4 kB in 0s (137 kB/s)
Selecting previously unselected package pigz.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../archives/pigz_2.4-1_amd64.deb ...
Unpacking pigz (2.4-1) ...
Setting up pigz (2.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
!wget https://traces1.inria.fr/oscar/files/Compressed/th_dedup.txt.gz

--2020-04-20 10:05:55--  https://traces1.inria.fr/oscar/files/Compressed/th_dedup.txt.gz
Resolving traces1.inria.fr (traces1.inria.fr)... 128.93.193.43
Connecting to traces1.inria.fr (traces1.inria.fr)|128.93.193.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3667953054 (3.4G) [application/gzip]
Saving to: ‘th_dedup.txt.gz’

th_dedup.txt.gz     100%[===================>]   3.42G  28.1MB/s    in 2m 11s  

2020-04-20 10:08:07 (26.6 MB/s) - ‘th_dedup.txt.gz’ saved [3667953054/3667953054]



In [0]:
!ls -al

In [0]:
# ลอง unzip ก่อน  ถ้าที่ไม่พอ ค่อยใช้วิธี gzip.open เอา
!pigz -d th_dedup.txt.gz

In [0]:
!ls -l   # 17 GB ก็โอเค รับได้  เขาว่า 370M คำ ไว้ต้องนับดู

total 16641192
drwxr-xr-x 1 root root        4096 Apr  3 16:24 sample_data
-rw-r--r-- 1 root root 17040570018 Apr 10  2019 th_dedup.txt


In [0]:
!ln -s th_dedup.txt th.txt

In [0]:
!head th.txt

ประกาศจังหวัดตาก เรื่อง เจตจำนงสุจริตในการบริหารงานของผู้ว่าราชการจังหวัดตาก ประกาศ ณ วันที่ ๒๖ เมษายน ๒๕๖๑ | “17 กรกฎาคม 2561 วันครบรอบ 4 ปี ศูนย์ดำรงธรรมจังหวัดตาก” ตามประกาศคณะรักษาความสงบแห่งชาติ ฉบับที่ 96/2557 ศูนย์ดำรงธรรมจังหวัดตาก | "คำแนะนำเกี่ยวกับรายละเอียดการปฏิบัติในการลงทะเบียนเพื่อสวัสดิการแห่งรัฐเพิ่มเติมภายใต้โครงการไทยนิยม ยั่งยืน"
-- ส่วนราชการ -- สำนักงานจังหวัดตาก สนง.โยธาธิการและผังเมืองจังหวัดตาก สนง.พัฒนาชุมชนจังหวัดตาก สำนักงานเกษตรและสหกรณ์จังหวัดตาก สำนักงานปฏิรูปที่ดินจังหวัดตาก สำนักงานตรวจบัญชีสหกรณ์ตาก แขวงการทางตากที่ 1 แขวงการทางตากที่ 2 ศูนย์พัฒนาฝีมือแรงงานจังหวัดตาก สนง.เขตพื้นที่การศึกษาตาก เขต 1 ศูนย์การศึกษานอกโรงเรียนจังหวัดตาก รพ.แมสอด สนง.อุตสาหกรรมจังหวัดตาก เรือนจำกลางตาก สำนักงานสถิติจังหวัดตาก สถานีวิทยุกระจายเสียงแห่งประเทศไทยจังหวัดตาก สถานีวิทยุกระจายเสียงแห่งประเทศไทยแม่สอด ด่านตรวจคนเข้าเมืองตาก สำนักงานพาณิชย์จังหวัดตาก สำนักงานสรรพากรพื้นที่ตาก หอการค้าจังหวัดตาก สำนักงานสหกรณ์จังหวัดตาก ศูนย์วิจัยและพัฒนาประมงน้ำจืดตาก สนง.ปศุสัตว์

## EDA

In [0]:
# อ่านอย่างเดียว นานไหม?  5 นาทีก็โอเคนะ
%%time
for line in open('th.txt'):
  pass

CPU times: user 1min 6s, sys: 6.9 s, total: 1min 13s
Wall time: 5min 13s


In [0]:
# wc ยังเกือบ 5 นาทีเลย.  22 ล้านบรรทัด
%%time
!wc -l th.txt

22856734 th.txt
CPU times: user 621 ms, sys: 96.2 ms, total: 717 ms
Wall time: 4min 55s


In [0]:
# ถ้า 1M ก็ 300/22 ≈ 10 sec
%%time
for i, line in enumerate(open('th.txt')):
  if i==1_000_000: break
  pass

CPU times: user 2.46 s, sys: 223 ms, total: 2.69 s
Wall time: 2.69 s


In [0]:
linelen = []
for line in open('th.txt'):
  n = len(line)
  if n>200: continue
  linelen.append(n)
  if len(linelen)==100_000: break

In [0]:
# อย่างน้อยต้อง 100 char ทุกอัน
px.histogram(x=linelen, nbins=20, width=600)

In [0]:
# มีสักกี่ % ที่มีภาษาอังกฤษปน?
pat = re.compile('[a-zA-Z]')
f.seek(0)
eng = 0
for i in range(1_000_000):
  if pat.search(f.readline()):
    eng += 1
print(f"{eng} = {eng/10000:.2f} %")  # ครึ่งนึง

484743 = 48.47 %


## ตัดคำ

In [0]:
# อันไหนตัดคำเร็วสุด
from pythainlp import word_tokenize

In [0]:
# เลือกมา 1 แสน ที่มีแต่อักษรไทย และไม่ยาวเกินไป
lines = []
f.seek(0)
for line in f:
  if pat.search(line): continue
  if len(line)>500: continue
  lines.append(line)
  if len(lines)==100_000: break
len(lines)

100000

In [0]:
line = lines[0]  # บรรทัดแรก ก็โอเค
print(len(line), line)

353 ประกาศจังหวัดตาก เรื่อง เจตจำนงสุจริตในการบริหารงานของผู้ว่าราชการจังหวัดตาก ประกาศ ณ วันที่ ๒๖ เมษายน ๒๕๖๑ | “17 กรกฎาคม 2561 วันครบรอบ 4 ปี ศูนย์ดำรงธรรมจังหวัดตาก” ตามประกาศคณะรักษาความสงบแห่งชาติ ฉบับที่ 96/2557 ศูนย์ดำรงธรรมจังหวัดตาก | "คำแนะนำเกี่ยวกับรายละเอียดการปฏิบัติในการลงทะเบียนเพื่อสวัสดิการแห่งรัฐเพิ่มเติมภายใต้โครงการไทยนิยม ยั่งยืน"



In [0]:
%%time 
for i, line in enumerate(lines):
  words = word_tokenize(line)
  print('\r', i, end='')

 99999CPU times: user 1min 26s, sys: 8.92 s, total: 1min 35s
Wall time: 1min 26s


In [0]:
# icu เร็วกว่าเท่านึง  ก็โอเคนะ เลือกตัวเร็วสุดละกัน
%%time 
for i, line in enumerate(lines):
  words = icu_segment(line)
  print('\r', i, end='')

 99999CPU times: user 31.6 s, sys: 8.43 s, total: 40 s
Wall time: 30.9 s


In [0]:
# icu generator ยิ่งเร็ว
%%time 
for i, line in enumerate(lines):
  gen = icu_gen_words(line)
  print('\r', i, end='')

 99999CPU times: user 21.7 s, sys: 7.39 s, total: 29.1 s
Wall time: 20.4 s


In [0]:
words

['5',
 '.',
 ' ',
 'ปักกิ่ง',
 ' ',
 ':',
 ' ',
 'ทัวร์',
 'ชม',
 'หอ',
 'สัก',
 'กา',
 'ระฟ้า',
 'และ',
 'เรียน',
 'รำ',
 'ไท',
 'เก็ก',
 'ครึ่ง',
 'วัน',
 'ใน',
 'ปักกิ่ง',
 ',',
 ' ',
 'บัตร',
 'เข้า',
 'ชม',
 'การ',
 'แสดง',
 'อุปรากร',
 'จีน',
 ' ',
 '(',
 'งิ้ว',
 ')',
 ' ',
 'ที่',
 'โรง',
 'ละคร',
 'ลี่',
 'หยวน',
 ',',
 ' ',
 'ทัวร์',
 'กำแพง',
 'เมือง',
 'จีน',
 ' ',
 'ด่า',
 'นมู่เถียนย',
 'วี่',
 'ด้วย',
 'รถ',
 'บัส',
 '\n']

## Word Frequency

เอาคร่าวๆ ก็พอ ใช้ icu ตัดเร็วหน่อย ผิดไม่เป็นไร

In [0]:
%%time
cnt = Counter()
for line in lines:
  cnt.update(icu_gen_words(line))

CPU times: user 5.71 s, sys: 0 ns, total: 5.71 s
Wall time: 5.72 s


In [0]:
cnt.most_common(10)

[(' ', 901203),
 ('ที่', 122491),
 ('\n', 99936),
 ('การ', 75076),
 ('ใน', 66595),
 ('และ', 66409),
 ('ได้', 59123),
 ('เป็น', 58051),
 ('มี', 55223),
 ('จะ', 53795)]

1 แสน = 5 วิ. 

22 ล้าน => 22*50 ≈ 1000 วิ ≈ 20 นาที

In [0]:
# งั้นทำกับ data ทั้งหมดเลยดู  32 นาทีก็พอทนไหว
%%time
cnt = Counter()
f.seek(0)
for i, line in enumerate(f):
  cnt.update(icu_gen_words(line))
  if i%10000==0: 
    print(f'\r{ i/1e6 :.1f} M', end='')

 22850000CPU times: user 31min 53s, sys: 25.6 s, total: 32min 19s
Wall time: 32min 23s


In [0]:
print(len(cnt)) # มี vocab กี่คำ... 6 ล้าน
sum(cnt.values())  # มีกี่ token 2 พันล้าน (นับ punctuation ด้วย)

6003086


1965782499

In [0]:
df = pd.DataFrame(cnt.most_common(20), columns=['word', 'freq']); df

,word,freq
0,,344205219
1,ที่,36117260
2,การ,24289761
3,\n,22804893
4,ใน,20490595
5,และ,19771264
6,ของ,17282528
7,ได้,17035265
8,.,16833381
9,เป็น,16192075


In [0]:
df = pd.DataFrame(cnt.most_common(), columns=['word', 'freq'])
df.to_csv('oscar_word_freq.csv', index=False)  # 95 MB

In [0]:
!head oscar_word_freq.csv

word,freq
 ,344205219
ที่,36117260
การ,24289761
"
",22804893
ใน,20490595
และ,19771264
ของ,17282528
ได้,17035265


In [0]:
# copy เข้า gdrive ไว้ให้ load ง่ายๆ
!cp oscar_word_freq.csv "drive/My Drive/"

In [0]:
!gdown https://drive.google.com/uc?id=1Xik_9IpasiuW5MXyiAA4UcmbRm0jLcj5

Downloading...
From: https://drive.google.com/uc?id=1Xik_9IpasiuW5MXyiAA4UcmbRm0jLcj5
To: /content/oscar_word_freq.csv
99.8MB [00:00, 239MB/s]


In [0]:
pd.read_csv('oscar_word_freq.csv').head()

,word,freq
0,,344205219
1,ที่,36117260
2,การ,24289761
3,\n,22804893
4,ใน,20490595
